In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Concatenate, Conv1D, Activation, TimeDistributed, Flatten, RepeatVector, Permute,multiply
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, GlobalAveragePooling1D, MaxPooling1D, SpatialDropout1D, BatchNormalization
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.layers.core import K
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/home/brian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11742289275102371150
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 1869021184
locality {
  bus_id: 1
}
incarnation: 17979890597722925516
physical_device_desc: "device: 0, name: GeForce GTX 970M, pci bus id: 0000:01:00.0"
]


/home/brian/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


#### some helper stuff for tracking performance over the duration of the project

In [2]:
def write_model_timestamp(model_type, kfolds, scores, note):
    '''
    Parameters:
    model_type = string description of the model(s) used to make these scores
    kfolds     = how many folds in kfold cross validation used
    scores     = list of ROC AUC avg scores of models for each class, floats should be like 0.9784
    note       = string, whatever is of note about the model, made a change or whatever
    
    Returns:
    None, but writes (appends) a line to scores.txt in the root directory so that progress can be tracked
    The format is:
            time(s)~model_type~kfold~avg_roc_auc~toxic_auc~s_toxic_auc~obscene_auc~threat_auc~insult_auc~i_hate_auc~notes
            
    scores.txt is a tilde '~' seperated CSV like:
        time~model_type~kfold~avg_roc_auc~toxic_auc~s_toxic_auc~obscene_auc~threat_auc~insult_auc~i_hate_auc~notes
        1520303252~0.9794005980274005~note something
    '''

    out_text = "{:10.0f}~{:}~{:2d}~{:0.8f}~{:0.8f}~{:0.8f}~{:0.8f}~{:0.8f}~{:0.8f}~{:0.8f}~{:}\n".format(time.time(), 
                                             model_type, 
                                             kfolds, 
                                             np.mean(scores),
                                             scores[0],
                                             scores[1],
                                             scores[2],
                                             scores[3],
                                             scores[4],
                                             scores[5],                                                
                                             note)
    
    with open("../scores.txt", 'a') as out_file:
        out_file.write(out_text)
        
        print("wrote:")
        print(out_text)
        print("to file")

#### Load data and light processing

In [3]:
train = pd.read_csv('../data/train.csv').fillna(' ')
test = pd.read_csv('../data/test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

#### Vectorize words from both corpora

In [4]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=25000)    # 10k was initial

word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=25000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [5]:
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

print("train shape:", train_word_features.shape)
print("test shape:", test_word_features.shape)

train shape: (159571, 25000)
test shape: (153164, 25000)


#### Make a neural net from scratch, what could possibly go wrong

In [6]:
def dnn_model(learning_rate=0.01, lr_decay=1e-6, drop_out=0.2, input_shape=(50000,)):
    
    DROPOUT = drop_out

    # Input
    _inputs = Input(shape=input_shape)

    # Try like 5 hidden layers starting big and getting small
    
    # maybe try kernel_initializer='he_normal' on dense layers
    X = Dense(256, activation="relu", name="dense_1")(_inputs)
    X = BatchNormalization()(X)
    X = Dropout(DROPOUT)(X)

    X = Dense(128, activation="relu", name="dense_2")(X)
    X = BatchNormalization()(X)
    X = Dropout(DROPOUT)(X)

    X = Dense(64, activation="relu", name="dense_3")(X)
    X = BatchNormalization()(X)
    X = Dropout(DROPOUT)(X)

    X = Dense(32, activation="relu", name="dense_4")(X)
    X = BatchNormalization()(X)
    X = Dropout(DROPOUT)(X)

    X = Dense(16, activation="relu", name="dense_5")(X)
    X = BatchNormalization()(X)
    X = Dropout(DROPOUT)(X)

    _outputs = Dense(1, activation='sigmoid', name="output_1")(X)

    # gather model
    model = Model(inputs=_inputs, outputs=_outputs, name="bad_dnn")
    
    # configure optimizer
    optimizer = Adam(lr=learning_rate, decay=lr_decay)
    
    # compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model
    
# https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,6)),str(round(roc_val,6))),end=100*' '+'\n')
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

#### Use a single fold for experimentation since 10fold will take forever

In [ ]:

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# for _class in class_names:
x_train, x_test, y_train, y_test = train_test_split(train_word_features, 
                                                    train[class_names[3]], 
                                                    test_size=0.1, 
                                                    random_state=1337)

print(x_train.shape)
print(class_names[3])

model = dnn_model(learning_rate=0.00025, drop_out=0.50, input_shape=(x_train.shape[1],))
# print(model.summary())

hist = model.fit(x_train, y_train,
                batch_size=512,
                epochs=7,
                verbose=1,
                validation_data=(x_test, y_test),
                callbacks=[roc_callback(training_data=(x_train, y_train),validation_data=(x_test, y_test))])



# for _class in class_names:
#     x_train, x_test, y_train, y_test = train_test_split(train_word_features, train[_class], test_size=0.1, random_state=1337)


#     results = []
    
# #     print("Class: {:}".format(_class))
# #     for nest in nests:
#     model1 = RandomForestClassifier(n_estimators=32, n_jobs=-1, max_depth=512)
#     model2 = MultinomialNB(alpha=0.03, fit_prior=False)
#     model3 = LogisticRegression(solver='sag')
    
#     meta_model = VotingClassifier(estimators=[('rf', model1), ('mnb', model2), ('lr', model3)],
#                                   weights=[1.0, 1.0, 1.5],
#                                   voting='soft',
#                                   n_jobs=-1)
    
#     meta_model.fit(x_train, y_train)

#     preds = meta_model.predict(x_test)

#     result = roc_auc_score(y_test, preds)
#     results.append(result)

#     print("Class: {: <14}  ROC AUC: {:0.4f}".format(_class, result))

(143613, 25000)
threat
Train on 143613 samples, validate on 15958 samples
Epoch 1/7
143613/143613 [==============================] - 31s 215us/step - loss: 0.9127 - acc: 0.5278 - val_loss: 0.6340 - val_acc: 0.7144
Epoch 2/7
143613/143613 [==============================] - 34s 237us/step - loss: 0.6253 - acc: 0.6765 - val_loss: 0.4496 - val_acc: 0.9959
Epoch 3/7
143613/143613 [==============================] - 34s 234us/step - loss: 0.4504 - acc: 0.8475 - val_loss: 0.3406 - val_acc: 0.9961
Epoch 4/7
143613/143613 [==============================] - 34s 234us/step - loss: 0.3154 - acc: 0.9514 - val_loss: 0.2321 - val_acc: 0.9967
Epoch 5/7
143613/143613 [==============================] - 34s 235us/step - loss: 0.2119 - acc: 0.9833 - val_loss: 0.1439 - val_acc: 0.9967
Epoch 6/7
 92160/143613 [==================>...........] - ETA: 11s - loss: 0.1559 - acc: 0.9904

In [ ]:
scores = []

NUM_FOLDS = 10

train_features = train_word_features.copy()

# submission = pd.DataFrame.from_dict({'id': test['id']})

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

for class_name in class_names:
    train_target = train[class_name]
    
    model1 = RandomForestClassifier(n_estimators=32, n_jobs=-1, max_depth=512)
    model2 = MultinomialNB(alpha=0.03, fit_prior=False)
    model3 = LogisticRegression(solver='sag')
    
    classifier = VotingClassifier(estimators=[('rf', model1), ('mnb', model2), ('lr', model3)],
                                  voting='soft',
                                  n_jobs=-1)

    kfold = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=1337)
    
#     results = cross_val_score(classifier, train_features, train_target, cv=5, n_jobs=-1, scoring='roc_auc')
    results = cross_val_score(classifier, train_features, train_target, cv=kfold, scoring='roc_auc')
    
    print('CV Spread for class "{}":'.format(class_name))
    for result in results:
        print("    {:0.4f}".format(result), end=" ")
        
    print(" ")
        
    cv_score = np.mean(results)
    scores.append(cv_score)
    
    print('    CV score for class "{}" is {:0.4}\n'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
#     submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {:0.4f}'.format(np.mean(scores)))

write_model_timestamp('DNN', NUM_FOLDS, scores, "word2vec max 25k features, 5 FC layers 256->128->64>32->16->1 output")